<a href="https://colab.research.google.com/github/PHoepner/big-sleep/blob/main/Looped_Gif_Creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Just enter your text in the form below and select Runtime-> Runall. 

It will play a pleasant beep and attempt to download the file.

You should be able to just run this as-is. I can try to answer questions at https://www.reddit.com/user/exquisite_corpsed 

In [ ]:
#@title Text to dream from
DreamText = "Do Androids dream of Electric Sheep" #@param {type:"string"}
DreamText2 = "A painting of a squirrel by picasso" #@param {type:"string"}


In [ ]:
!git clone https://github.com/phoepner/big-sleep.git
!pip install ftfy
!pip install boto3
!pip install einops
!apt install gifsicle
import subprocess
for _ in range(2):
    CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
    print("CUDA version:", CUDA_version)

    if CUDA_version == "10.0":
        torch_version_suffix = "+cu100"
    elif CUDA_version == "10.1":
        torch_version_suffix = "+cu101"
    elif CUDA_version == "10.2":
        torch_version_suffix = ""
    else:
        torch_version_suffix = "+cu110"

    !pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex



In [ ]:
%cd /content/big-sleep
SearchText = "\"" + DreamText + "\""
!python gen_gif.py $SearchText\

SearchText2 = "\"" + DreamText2 + "\""
!python gen_gif.py $SearchText2\

In [ ]:
from google.colab import files
from google.colab import output

fileName = DreamText.replace(" ", "_")
fileName2 = DreamText2.replace(" ", "_")

FileStartIndex = 250
NumberOfFrames = 100
FirstPathIndex =  250
SecondPathIndex =  0
!python /content/big-sleep/interpolation.py --lat1 /content/big-sleep/$fileName\.$FirstPathIndex\.pth --lat2 /content/big-sleep/$fileName2\.$SecondPathIndex\.pth --steps $NumberOfFrames --startFileIndex $FileStartIndex --fileName $fileName

FileStartIndex = 250
NumberOfFrames = 100
FirstPathIndex =  250
SecondPathIndex =  0
!python /content/big-sleep/interpolation.py --lat1 /content/big-sleep/$fileName2\.$FirstPathIndex\.pth --lat2 /content/big-sleep/$fileName\.$SecondPathIndex\.pth --steps $NumberOfFrames --startFileIndex $FileStartIndex --fileName $fileName2

!ffmpeg -start_number 0 -i $fileName\.%d.png $fileName\.gif
!ffmpeg -start_number 0 -i $fileName2\.%d.png $fileName2\.gif

!gifsicle $fileName\.gif $fileName2\.gif > combo.gif
files.download("combo.gif")

output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

